In [ ]:
import pandas as pd
import numpy as np
import re
import time
from tqdm import tqdm
from util import pre_process
import pickle
# tqdm.pandas()
%matplotlib inline
from nltk import word_tokenize,pos_tag

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

path='./data/'

In [ ]:
train=pd.read_csv(path+'train_release.csv')
# test=pd.read_csv(path+'validation.csv')
test=pd.read_csv(path+'test.csv')
canditate=pd.read_csv(path+'candidate_paper_for_wsdm2020.csv')
train=train[~train['description_id'].isnull()]

In [ ]:
train_pre=pd.read_csv(path+'train_pre.csv')
test_pre=pd.read_csv(path+'stage2_test_pre.csv')
canditate_pre=pd.read_csv(path+'candidate_paper_pre.csv')

In [ ]:
train_pre['label']=1
# train_pre=train_pre.drop_duplicates('description_id')
train_pre.loc[(train_pre['description_id']=='6.45E+04')&(train_pre['paper_id']=='55a461c3612ca64868949e8e'), 'description_id'] = '6.45E+05'


In [ ]:
# '获取召回数据函数'
def get_recall(path):
#     data=pd.read_csv(path,header=None,names=['description_id']+[i for i in range(1,21)]).drop_duplicates('description_id').values
    data=pd.read_csv(path,header=None,names=['description_id']+[i for i in range(1,21)]).values
    description_id = []
    pid = []
    for i in data:
        description_id.extend([i[0]] * 20)
        pid.extend(list(i[1:]))
    return  pd.DataFrame({'description_id':description_id,'pid':pid})

def get_recall2(path):
#     data=pd.read_csv(path,header=None,names=['description_id']+[i for i in range(1,21)]).drop_duplicates('description_id').values
    data=pd.read_csv(path).values
    description_id = []
    pid = []
    for i in data:
        description_id.extend([i[0]] * 20)
        pid.extend(list(i[1:21]))
    return  pd.DataFrame({'description_id':description_id,'pid':pid})

In [ ]:
# '召回pid'
re_path='./队友召回结果/test_tfidf_200.csv'
test_recall=get_recall2(re_path)
test_data=test_pre.merge(test_recall,on='description_id',how='inner')

In [ ]:
test_data = test_data[test_data['pid'].notnull()].reset_index(drop=True)

In [ ]:
re_path='train_pairs_20number_tfidf2.csv'
train_recall=get_recall(re_path)
train_recall.loc[[i for i in range(1177020, 1177020+20)], 'description_id'] = '6.45E+05'
train_data=train_pre.merge(train_recall,on='description_id',how='inner')
train_data['label']=list(map(lambda x,y: int(x==y),train_data['pid'],train_data['paper_id']))

In [ ]:
'获取召回数据函数'
def get_recall(path):
#     data=pd.read_csv(path,header=None,names=['description_id']+[i for i in range(1,21)]).drop_duplicates('description_id').values
    data=pd.read_csv(path,header=None,names=['description_id']+[i for i in range(1,401)]).values
    description_id = []
    pid = []
    for i in data:
        description_id.extend([i[0]] * 100)
        pid.extend(list(i[1:101]))
    return  pd.DataFrame({'description_id':description_id,'pid':pid})

def get_recall2(path):
#     data=pd.read_csv(path,header=None,names=['description_id']+[i for i in range(1,21)]).drop_duplicates('description_id').values
    data=pd.read_csv(path).values
    description_id = []
    pid = []
    for i in data:
        description_id.extend([i[0]] * 100)
        pid.extend(list(i[1:101]))
    return  pd.DataFrame({'description_id':description_id,'pid':pid})

In [ ]:
'召回pid'
re_path='队友召回结果/test_bm25_200.csv'
test_recall2=get_recall2(re_path)
test_data2=test_pre.merge(test_recall2,on='description_id',how='inner')
test_data2 = test_data2[test_data2['pid'].notnull()].reset_index(drop=True)

In [ ]:
train_recall2[(train_recall2['description_id']=='6.45E+04')&(train_recall2.index>4000000)]

In [ ]:
re_path='train_pairs_100number2_scores.csv'
train_recall2=get_recall(re_path)
# train_recall2.loc[[i for i in range(1177020, 1177020+20)], 'description_id'] = '6.45E+05'  #20
# train_recall2.loc[[i for i in range(2354040, 2354040+40)], 'description_id'] = '6.45E+05'  # 40
# train_recall2.loc[[i for i in range(4708080, 4708080+80)], 'description_id'] = '6.45E+05'  # 80
train_recall2.loc[[i for i in range(5885100, 5885100+100)], 'description_id'] = '6.45E+05'  # 100
# train_recall2.loc[[i for i in range(5885100, 5885100+200)], 'description_id'] = '6.45E+05'  # 200
train_data2=train_pre.merge(train_recall2,on='description_id',how='inner')
train_data2['label']=list(map(lambda x,y: int(x==y),train_data2['pid'],train_data2['paper_id']))

In [ ]:
train_data3 = pd.concat([train_data, train_data2], axis=0).drop_duplicates(subset=['description_id', 'paper_id', 'pid']).reset_index(drop=True)

In [ ]:
test_data3 = pd.concat([test_data, test_data2], axis=0).drop_duplicates(subset=['description_id', 'pid']).reset_index(drop=True)

In [ ]:
'删去没用被召回的数据'
temp=train_data3[['description_id','label']].groupby('description_id')['label'].apply(lambda x:len(set(list(x)))).reset_index()
temp.columns=['description_id','label']
ids=temp[temp['label']==2]['description_id'].values
train_data3=train_data3[train_data3['description_id'].isin(ids)]

In [ ]:
train_data3.label.sum() / train_pre.shape[0]

In [ ]:
train_data3 = train_data3.sample(frac=1, random_state=2019).reset_index(drop=True)

In [ ]:
canditate_pre.loc[canditate_pre['paper_id']=='5c13696bda56295a08a2db1c', 'keywords'] = 'Apoptosis;As;Aspartic acid;Cis-Dichlorodiammine platinum;Cr;Cu;Cyclophosphamide;Ginkgo biloba exocarp extracts;Hg;Lewis lung cancer;MAPK signaling pathways;Mannose;Pb;Rhamnose;Rhodamine 123;alanine;arabinose;galactose;galacturonic acid;glucose;glutamic acid;glycine;isoleucine;leucine;lysine;methionine;phenylalanine;proline;serine;threonine;tryptophan;valine'
canditate_pre.loc[canditate_pre['paper_id']=='59d053750cf2e9d7f1f2e6aa', 'keywords'] = '2-Aminobutyric acid;5-Hydroxyindoleacetic acid;Acetic acid;Angelica sinensis;Arabinitol;Aspartic acid;Blood deficiency syndrome;Butanoic acid;Cholesterol;D-glucose;Glucitol;Gluconic acid;Integrated Metabolomics;L-Aspartic acid;L-Proline;L-Valine;Metabolic pathway analysis;N-Acetylneuraminic acid;Processed products;Urea;fumaric acid;glycerol;glycylglycine;linoleic acid'
canditate_pre.loc[canditate_pre['paper_id']=='5c0f7cb4da562944ac7de5cc', 'keywords'] = 'Acetic acid;Acrylamide;Ammonium sulfate;Aquaculture;Benzamidine;Benzoyl-arginine-p-nitroanilide;Citric acid;Dimethyl sulfoxide;Ferric chloride;Ferrous chloride;Fish processing residues;Glutaraldehyde;Glycine;Hydrochloric acid;Immobilization and enzyme reuse;Monosodium phosphate;Potassium permanganate;Protease inhibitors;Sodium bisulfite;Sodium chloride;Sodium hydroxide;Sodium phosphate dibasic;Tris 2-Amino-2-hydroxymethyl-propane-1,3-diol'
canditate_pre.loc[canditate_pre['paper_id']=='57da02330cf2ce2e6b0f19df', 'keywords'] = 'Nomenclature;One fungus one name;aurorae;bragantina;candida;cantabriensis;dentifunda;dombeyi;epigloea;eucalyptigena;eucastaneae;euskadiensis;fumea;fusiformis;gemella;gossypina;lunata;narcissi;nebularis;nigrograna;palmiculminata;phasma;polyporicola;prolifera;protea-sedis;rossii;stenoceras;thermara;zambiensis;Sporothrix abietina;Sporothrix schenckii;Sporotrichosis;Taxonomy'


In [ ]:
canditate_pre.loc[canditate_pre['abstract']=='n o n e', 'abstract'] = 'none'

In [ ]:
canditate_pre['title_pro'] = canditate_pre['title'].fillna('none')
canditate_pre['abstract_pre'] = canditate_pre['abstract'].fillna('none')
canditate_pre['keywords'] = canditate_pre['keywords'].fillna('none')

In [ ]:
canditate_pre['text_b'] = canditate_pre['title_pro'] + ' '+ canditate_pre['abstract_pre']+' '+canditate_pre['keywords'].apply(lambda x: x.replace(';',' '))

In [ ]:
train_copy = train_data3.copy()

In [ ]:
train_data3 = pd.merge(train_data3, canditate_pre[['pid', 'text_b']], on='pid', how='left')
train_data3.rename(columns={'key_text': 'text'}, inplace=True)
train_data3 = train_data3[train_data3['text'].notnull()].reset_index(drop=True)

tmp = train_data3[['text', 'pid', 'label']]
tmp = tmp.groupby(['text', 'pid'])['label'].agg('mean').reset_index()
tmp = tmp[(tmp['label']>0)&(tmp['label']<1)][['text', 'pid']].reset_index(drop=True)
tmp['one'] = 1
train_data3 = pd.merge(train_data3, tmp, on=['text', 'pid'], how='left')
train_data3['label'] = train_data3.apply(lambda x: 1 if x['one'] == 1 else x['label'], axis=1)
del train_data3['one'], tmp
print(train_data3.shape)

train_data3 = train_data3.drop_duplicates(subset=['text', 'pid', 'label']).drop_duplicates().reset_index(drop=True)
print(train_data3.shape)

train_data3[['description_id', 'pid', 'text', 'text_b', 'label']].to_csv('new_data/train_data3_bm2540_tfidf20_6.csv', index=False)

In [ ]:
tmp = train_copy[train_copy['label']==1][['paper_id', 'key_text']].drop_duplicates()
tmp = tmp.groupby('paper_id')['key_text'].apply(lambda x: ' '.join(list(x))).reset_index()
tmp['one'] = 1

In [ ]:
canditate_pre = pd.merge(canditate_pre, tmp, on='paper_id', how='left')
canditate_pre['text_b'] = canditate_pre.apply(lambda x: x['key_text'] + ' ' + x['text_b'] if x['one']==1 else x['text_b'], axis=1)
del canditate_pre['key_text'], canditate_pre['one']

canditate_pre.rename(columns={'paper_id': 'pid'}, inplace=True)

In [ ]:
test_data3 = pd.merge(test_data3, canditate_pre[['pid', 'text_b']], on='pid', how='left')
test_data3.rename(columns={'key_text': 'text'}, inplace=True)

# 召回80用这个
test_data3 = test_data3.fillna('none')
# 召回90-80用这个
# test_data3 = test_data3[test_data3['key_text_pre'].notnull()].reset_index(drop=True)

test_data3[['description_id', 'pid', 'text', 'text_b']].to_csv('new_data/stage2_test_data3_bm2580_tfidf20_8.csv', index=False)

剩余召回数据集

In [ ]:
tmp1 = pd.read_csv('./new_data/stage2_test_data3_bm2580_tfidf20_8.csv')
tmp2 = pd.read_csv('./new_data/stage2_test_data3_bm2590_tfidf20_10.csv')

tmp1 = pd.concat([tmp1, tmp2]).reset_index(drop=True)
del tmp2

tmp1 = tmp1[['description_id', 'pid']]
tmp1['one'] = 1
test_data3 = pd.merge(test_data3, tmp1, on=['description_id', 'pid'], how='left')

test_data3 = test_data3[test_data3['one']!=1].reset_index(drop=True)
del test_data3['one']

canditate = pd.read_csv('./data/candidate_paper_for_wsdm2020.csv')
canditate.rename(columns={'paper_id': 'pid'}, inplace=True)
test_data3 = pd.merge(test_data3, canditate[['pid', 'journal']], on='pid', how='left')

test_data3 = test_data3[test_data3['journal']!='no-content'].reset_index(drop=True)

test_data3[['description_id', 'pid', 'text', 'text_b']].to_csv('new_data/stage2_test_data3_bm2590_tfidf20_10.csv', index=False)

# 保存发给队友
# tmp = test_data3[['description_id', 'pid']]
# tmp.columns = ['description_id', 'paper_id']
# tmp.to_csv('new_data/stage2_test_data3_bm2590_tfidf20_10_toxiong.csv', index=False)

tmp.shape